In [150]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import statsmodels.api as sm

In [151]:
data = pd.read_csv('data2021_2023.csv', encoding='cp949')

In [152]:
data['모기 개체수'] = data['모기 개체수'].str.replace(',', '').astype(int)

In [153]:
features = [
    '평균기온', '최저기온','일강수량(mm)', '평균 풍속(m/s)',
     '최소 상대습도(%)','평균 상대습도(%)', '평균 현지기압(hPa)', '평균 해면기압(hPa)',
    '합계 일조시간(hr)', '평균 전운량(1/10)', '1일전 모기 개체수','2일전 모기 개체수','3일전 모기 개체수']

target = '모기 개체수'

In [154]:
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [155]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [156]:
X_train_scaled_ = sm.add_constant(X_train_scaled)
X_test_scaled_ = sm.add_constant(X_test_scaled)

In [157]:
model = sm.OLS(y_train, X_train_scaled_)
results = model.fit()

In [158]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 모기 개체수   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     117.2
Date:                Tue, 28 May 2024   Prob (F-statistic):          5.84e-126
Time:                        23:27:20   Log-Likelihood:                -2949.5
No. Observations:                 405   AIC:                             5927.
Df Residuals:                     391   BIC:                             5983.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        788.5806    357.044      2.209      0.0

In [159]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [160]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [161]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 123939.0206
Train RMSE : 352.0497
Train R^2 : 0.7958
Performance for TEST--------
Test MSE : 99920.7896
Test RMSE : 316.1025
Test R^2 : 0.8245


In [162]:
# 선택된 특성들의 평균 계산
feature_means = data[features].mean().values.reshape(1, -1)

feature_means_scaled = scaler.transform(feature_means)

y_pred = model.predict(feature_means_scaled)

print(f"Predicted Mosquito Count(2021-2023): {y_pred[0]:.2f}")
print("-----------------------------")

Predicted Mosquito Count(2021-2023): 1989.02
-----------------------------


c:\Users\songs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


# 2021-2023 Ridge

In [163]:
from sklearn.linear_model import Ridge

In [164]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42)

In [165]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [166]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

alphas = [0.01, 0.1, 1, 10, 100, 1000]
train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    # Ridge 회귀 모델 생성 및 학습
    model = Ridge(alpha=alpha)
    model.fit(X_train_scaled, y_train)

    # 훈련 세트와 검증 세트에서의 예측 수행
    y_train_hat = model.predict(X_train_scaled)  
    y_val_hat = model.predict(X_val_scaled)  

    # R^2 값 계산
    train_r2 = r2_score(y_train, y_train_hat)
    val_r2 = r2_score(y_val, y_val_hat)
    
    # 결과 저장
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    # 결과 출력
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print('---------------------')


Alpha: 0.01
Train R^2: 0.7898
Validation R^2: 0.7915
---------------------
Alpha: 0.1
Train R^2: 0.7895
Validation R^2: 0.7924
---------------------
Alpha: 1
Train R^2: 0.7850
Validation R^2: 0.7919
---------------------
Alpha: 10
Train R^2: 0.7293
Validation R^2: 0.7516
---------------------
Alpha: 100
Train R^2: 0.3822
Validation R^2: 0.4169
---------------------
Alpha: 1000
Train R^2: 0.0633
Validation R^2: 0.0691
---------------------


In [167]:
model = Ridge(alpha=1) #1 채택
model.fit(X_train_scaled, y_train)

Ridge(alpha=1)

In [168]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [169]:
coefficients = pd.DataFrame({
    'Feature': features,
    'Ridge Coefficient': model.coef_
})
print(coefficients)

         Feature  Ridge Coefficient
0           평균기온         494.296180
1           최저기온         279.664194
2       일강수량(mm)          34.077684
3     평균 풍속(m/s)        -247.102346
4     최소 상대습도(%)        -249.746871
5     평균 상대습도(%)        -232.252409
6   평균 현지기압(hPa)         198.690456
7   평균 해면기압(hPa)         162.178593
8    합계 일조시간(hr)          -5.594553
9   평균 전운량(1/10)         121.861024
10    1일전 모기 개체수        2013.813037
11    2일전 모기 개체수         635.456830
12    3일전 모기 개체수         323.448527


In [170]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 136098.7586
Train RMSE : 368.9157
Train R^2 : 0.7850
Performance for TEST--------
Test MSE : 99326.6105
Test RMSE : 315.1612
Test R^2 : 0.8291


# 2021-2023 MinMax Scale, Lasso

In [171]:
from sklearn.linear_model import Lasso

In [172]:
alphas = [0.01, 0.1, 1, 10, 100]
train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    # Ridge 회귀 모델 생성 및 학습
    model = Lasso(alpha=alpha)
    model.fit(X_train_scaled, y_train)

    # 훈련 세트와 검증 세트에서의 예측 수행
    y_train_hat = model.predict(X_train_scaled)  
    y_val_hat = model.predict(X_val_scaled)  

    # R^2 값 계산
    train_r2 = r2_score(y_train, y_train_hat)
    val_r2 = r2_score(y_val, y_val_hat)
    
    # 결과 저장
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    # 결과 출력
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print('---------------------')

Alpha: 0.01
Train R^2: 0.7898
Validation R^2: 0.7914
---------------------
Alpha: 0.1
Train R^2: 0.7897
Validation R^2: 0.7915
---------------------
Alpha: 1
Train R^2: 0.7890
Validation R^2: 0.7918
---------------------
Alpha: 10
Train R^2: 0.7686
Validation R^2: 0.8025
---------------------
Alpha: 100
Train R^2: 0.4026
Validation R^2: 0.4431
---------------------


c:\Users\songs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.156e+07, tolerance: 1.918e+04
  model = cd_fast.enet_coordinate_descent(


In [173]:
model = Lasso(alpha=1) #1채택
model.fit(X_train_scaled, y_train)

Lasso(alpha=1)

In [174]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [175]:
coefficients = pd.DataFrame({
    'Feature': features,
    'Lasso Coefficient': model.coef_
})
print(coefficients)

         Feature  Lasso Coefficient
0           평균기온         698.134062
1           최저기온           0.000000
2       일강수량(mm)          -0.000000
3     평균 풍속(m/s)        -219.495946
4     최소 상대습도(%)        -143.914147
5     평균 상대습도(%)        -193.084323
6   평균 현지기압(hPa)         337.585422
7   평균 해면기압(hPa)           0.000000
8    합계 일조시간(hr)         -16.555110
9   평균 전운량(1/10)          59.649979
10    1일전 모기 개체수        2534.084758
11    2일전 모기 개체수         361.669299
12    3일전 모기 개체수         167.659612


In [176]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 133610.7008
Train RMSE : 365.5280
Train R^2 : 0.7890
Performance for TEST--------
Test MSE : 103542.0847
Test RMSE : 321.7796
Test R^2 : 0.8219
